# Format to CSV

In [ ]:
lines = []
with open(file="cawiki-20240501-pagelinks.sql", mode="r", encoding='utf-8') as f:
    for line in f:
        if line[:6] == "INSERT":
            for temp in line[32:-3].split("),("):
                    lines.append(temp.split(",")[:5])
lines[0]

In [3]:
import pandas as pd

In [ ]:
for i in range(6):
    df1 = pd.DataFrame(lines[10000000*i:10000000*(i+1)]) #total 65674867
    df1.to_csv("cawiki-pagelinks"+str(i)+".csv", index=False, header=None)
df1 = pd.DataFrame(lines[60000000:])
df1.to_csv("cawiki-pagelinks7.csv", index=False, header=None)

# Recover CSVs and concat

In [ ]:
import pandas as pd
import glob
 
# list all csv files only
csv_files = glob.glob('*.{}'.format('csv'))

import pandas as pd
 
# df_csv_concat = pd.concat([pd.read_csv(file) for file in csv_files ], ignore_index=True)
# df_csv_concat

df1 = pd.concat([pd.read_csv(csv_files[0], header=None),pd.read_csv(csv_files[1], header=None)])
df2 = pd.concat([pd.read_csv(csv_files[2], header=None),pd.read_csv(csv_files[3], header=None)])
df3 = pd.concat([pd.read_csv(csv_files[4], header=None),pd.read_csv(csv_files[5], header=None),pd.read_csv(csv_files[6], header=None)])
dfa = pd.concat([df1,df2,df3])

In [ ]:
dfa = dfa[dfa[1]==0] #65M -> 60M rows
dfa = dfa[dfa[3]==0] #60,8M -> 9,2M rows
dfa

In [ ]:
dfa.to_csv("cawiki-pagelinks.csv", index=False, header=None)

In [ ]:
# temp = list(df1[df1[4]=="427395"][0]) #tots els que apunten a !Kung_(grup_humà)
# for item in temp:
#     print(df1[df1[4]==item])

## Obrint page_data (page.sql) i convertint-lo a csv

In [1]:
lines = []
with open(file="cawiki-20240501-page.sql", mode="r", encoding='utf-8') as f:
    lines = f.readlines()
lines = lines[50:-12]

In [4]:
rows = []
for line in lines:
    line = line[26:]
    values = line.split("),(")
    values[0] = values[0][1:]
    values[-1] = values[-1][:-3]
    for value in values:
        rows.append(value.split(",")[:6])
page_df = pd.DataFrame(rows)[[0,1,2]]
page_df = page_df[page_df[1]=="0"][[0,2]] #namespace 0, drop 1
page_df.rename({0:"id",2:"name"},axis=1,inplace=True)
print(len(page_df),"rows fetched.")
page_df

1234381 rows fetched.


,id,name
0,1,'Àbac'
1,2,'Abadia'
2,8,'Adagi'
3,9,'Adam'
4,10,'Addicció'
...,...,...
1876243,2091651,'Helena_Paleòleg_(filla_de_Demetri_II_Paleòleg)'
1876245,2091653,'Margarida_Paleòleg'
1876246,2091654,'Anna_Paleologina_Cantacuzena'
1876247,2091656,'Anna_Paleologina'


In [ ]:
page_df

# The Great Replacement (literalment)

In [ ]:
repdic = page_df.set_index('id').T.to_dict('list')

In [ ]:
dfa[5] = dfa[0].apply(lambda x: repdic[str(x)][0][1:-1] if str(x) in repdic.keys() else "ERRmissing")

In [ ]:
dfa = dfa[[5,2]]
dfa.rename({5:"from",2:"to"},axis=1,inplace=True)
dfa["to"] = dfa["to"].apply(lambda x: x[1:-1])

In [ ]:
dfa

### Provant igraph

In [ ]:
import igraph as ig

In [ ]:
#preproc
vertices = set(list(dfa["from"]) + list(dfa["to"]))
edges = dfa.values

In [ ]:
print(len(vertices),len(edges))

In [ ]:
g = ig.Graph(directed=True)
g.add_vertices(vertices)
g.add_edges(edges)

In [ ]:
layout = g.layout("random")
ig.plot(g, layout = layout)